#imports

In [1]:
import nltk
nltk.download('punkt')
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/ece1786-project

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ece1786-project


In [2]:
import torch 
import numpy as np

from nltk.tokenize import sent_tokenize 

from pathlib import Path 
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from mingpt.bpe import BPETokenizer 
from mingpt.utils import set_seed 
from mingpt.model import GPT
import pandas as pd
set_seed(1234)

#Dataset class

In [3]:
class RecipeDataset(Dataset):
    def __init__(self,  truncation=-1):
        df = pd.read_pickle("cocktail_dataset.pkl")  
        recipes = []
        for i in range(len(df.index)):
            recipe = "RECIPE NAME\n" + df.loc[i, "Name"] + " \n\nRECIPE INGREDIENTS\n"
            for ingredient in df.loc[i, "Ingredients"]:
                recipe += ingredient + "\n"
            recipe += "\nRECIPE INSTRUCTIONS\n" 
            for instruction in  df.loc[i, "Instructions"]:
                recipe += instruction + "\n"
            recipes.append(recipe)

        # Tokenize
        self.tokenizer = BPETokenizer()
        self.data = []  # List of 1-d pytorch tensor
        for sent in recipes:
            tokenized = self.tokenizer(sent).view(-1)  # pytorch tensor
            if truncation >= 0:
                self.data.append(tokenized[:truncation])
            else:
                self.data.append(tokenized)

        # Count some items
        self.max_sentence_length = np.max([len(d) for d in self.data])

    def __len__(self):
        return len(self.data)

    def get_vocab_size(self):
        """
        We have to set this to the max vocab size (i.e., that decided by the BPE tokenizer), 
        but actually, only a small number of vocab is used, especially for the small text. 
        """
        return 50257

    def __getitem__(self, idx):
        """
        The output should be a tuple x and y, both as pytorch tensors.
        Please refer to the `run()` method in the mingpt/trainer.py script for 
        how the x and y are going to be used.
        """
        x = self.data[idx][:-1]
        y = self.data[idx][1:]
        return (x, y)

    def get_block_size(self):
        """
        block_size is the size at which lines are truncated to ensure they are equal-length.
        """
        return self.max_sentence_length
    

dataset = RecipeDataset(truncation=512) #use this for long

# Load model and eval

In [4]:
device = torch.device('cpu')

In [5]:
model_config = GPT.get_default_config()
model_config.model_type = 'gpt-nano'
model_config.vocab_size = dataset.get_vocab_size()
model_config.block_size = dataset.get_block_size()
model_config.n_classification_class = 2
model = GPT(model_config)
model.load_state_dict(torch.load("./recipe_baseline.pt", map_location=torch.device('cpu')))

number of parameters: 2.52M


<All keys matched successfully>

In [6]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import evaluate

In [11]:
def score(metric="bleu", filename="cocktail_dataset.pkl"):

  def create_propmts():
    df = pd.read_pickle(filename)
    answers = []
    prompts = []
    for i in range(len(df.index)):
      recipe = "RECIPE NAME\n" + df.loc[i, "Name"] + " \n\nRECIPE INGREDIENTS\n"
      prompts.append(recipe)
      for ingredient in df.loc[i, "Ingredients"]:
        recipe += ingredient + "\n"
      recipe += "\nRECIPE INSTRUCTIONS\n" 
      for instruction in  df.loc[i, "Instructions"]:
        recipe += instruction + "\n"
      if metric == "bleu":
        answers.append([recipe])
      else:
        answers.append(recipe)
    return answers, prompts

  answers, prompts = create_propmts()
  attempts = []
  for i in range(len(answers)):
    encoded_prompt = dataset.tokenizer(prompts[i]).to(device)
    generated_sequence = model.generate(encoded_prompt, device, temperature=0.7, max_new_tokens=100, do_sample=True)
    attempts.append(dataset.tokenizer.decode(generated_sequence[0].squeeze()))
  
  eval = evaluate.load(metric)
  results = eval.compute(predictions=attempts, references=answers)
  return results

In [12]:
score(metric="wer")

0.8491939677587104

In [13]:
score(metric="bleu")

{'bleu': 0.16861571351409319,
 'precisions': [0.5689924556939422,
  0.3306933381607531,
  0.21733743288513607,
  0.1498184864562971],
 'brevity_penalty': 0.6026845682677286,
 'length_ratio': 0.6638513513513513,
 'translation_length': 22401,
 'reference_length': 33744}

# examples

In [14]:
prompt = "RECIPE NAME\nWhiskey Sour \n\nRECIPE INGREDIENTS\n"
encoded_prompt = dataset.tokenizer(prompt).to(device)
generated_sequence = model.generate(encoded_prompt,device, temperature=0.7, max_new_tokens=100, do_sample=True)
print(dataset.tokenizer.decode(generated_sequence[0].squeeze()))

RECIPE NAME
Whiskey Sour 

RECIPE INGREDIENTS
1 1/2 ounces whiskey 
1/2 ounce lemon juice, freshly squeezed 
1/2 ounce egg white 
1/2 ounce or sherry 
1/2 ounce egg white

RECIPE INSTRUCTIONS
Add the gin, dry vermouth, vigorously dry-shake (such as a mixing glass) vigorously dry-shake (without ice) vigorously dry-shake (or ice) vigorously until well-chilled.
Strain into a rocks glass
